In [35]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# loading all data for model - text of the book, list of entities, and test text

with open("data/linear_vocab.txt",'r') as file_words:
    vocab = sorted([i.replace("•","").strip().lower() for i in file_words.read().split("\n") if i], key=lambda x: -len(x))
    vocab = [[i[:i.rindex(" ")], i[i.rindex(" ")+1:]] for i in vocab]
    
with open("data/Strang-Linear Algebra.txt",'r') as file_text:
    book = file_text.read().lower()

with open("data/new_book.txt",'r',encoding='windows-1252') as file_text:
    book2 = file_text.read().lower()
    
with open("data/test.txt") as filee:
    test_text = filee.read()

with open("data/linear_.txt",'w') as file_words:
    file_words.write("\n".join(sorted([" ".join(i) for i in vocab])))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
vocab[:5]

[['closure under addition and scalar multiplication', 'act'],
 ['negative semidefinite quadratic form', 'typ'],
 ['positive semidefinite quadratic form', 'typ'],
 ['general solution to a linear system', 'obj'],
 ['orthogonally diagonalizable matrix', 'mtrx']]

***Example of Spacy formatted data***

***EXAMPLE_DATA = [
    ("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
    ("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
]***


In [4]:
def book_into_spacy_format(book):
  data = []
  sentences = []
  for sentence in book.split("."):  
    sentence = sentence.replace("\n"," ").replace("¤","").replace("§","").strip()
    if len(sentence) > 25 and list(sentence).count(" ") < len(sentence)/3:
      words = []
      sentence = " ".join(sentence.split())
      taken = []
      for word in vocab:
        index = sentence.find(word[0])
        if index != -1 and all(i not in taken for i in range(index,index+len(word[0]))) and (len(sentence) <= index+len(word[0])+2 or not sentence[index+len(word[0])].isalpha() or sentence[index+len(word[0])] == "s") :
          words.append([index, index+len(word[0]),word[1]])
          taken.extend([i for i in range(index, index+len(word[0]))])
      for word in vocab:
        index = sentence.rfind(word[0])
        if index != -1 and all(i not in taken for i in range(index,index+len(word[0]))) and (len(sentence) <= index+len(word[0])+2 or not sentence[index+len(word[0])].isalpha() or sentence[index+len(word[0])] == "s") :
          words.append([index, index+len(word[0]),word[1]])
          taken.extend([i for i in range(index, index+len(word[0]))])
        # elif index != -1:
          # print(word, " - proposed ENTITY not found in the book")
      if words:
        data.append((sentence, {"entities": words}))
      sentences.append(sentence)
  return data, sentences


In [33]:
data, sentences = book_into_spacy_format(book + " " + book2)
test, sentences_test = book_into_spacy_format(test_text)

print("\nExample sentence and its entities\n", data[0])
print("\nExample sentence and its entities, in test data\n", test[0])


Example sentence and its entities
 ('0 0 contents preface iv 1 matrices and gaussian elimination 1 1', {'entities': [[39, 59, 'act']]})

Example sentence and its entities, in test data
 ('In this section, we will define the product of two matrices and state the basic properties of the resulting matrix algebra', {'entities': [[36, 43, 'act'], [108, 114, 'obj']]})


In [44]:
# finding other terms not in vocabulary among words in the books

def find_more_words(sent, vocab):
    for i in vocab:
      sent = sent.replace(i[0],"")
    words = []
    stop_words = set(stopwords.words('english')) 
    pos_data = nltk.ConditionalFreqDist((word.lower(), tag) for (word, tag) in nltk.pos_tag(nltk.word_tokenize(sent)))
    for word in pos_data.conditions():
      if len(word) > 3 and word.isalpha() and word not in stop_words:
        tags = pos_data[word].keys()
        words.append([word, ' '.join(tags)])
    return words

In [45]:
find_more_words(book2, vocab)[:10]

[['alger', 'NN'],
 ['springer', 'NN'],
 ['undergraduate', 'JJ'],
 ['mathematics', 'NNS'],
 ['eries', 'NNS'],
 ['jorg', 'VBP NN'],
 ['liesen', 'JJ NN VBN'],
 ['olker', 'NN'],
 ['mehrmann', 'NN NNP VBD'],
 ['series', 'NN']]

In [11]:
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


In [17]:
def create_and_train_model(train_data, output_dir, n_iter,model=None):
    if model is not None:
        nlp = spacy.load(model)  # load existing model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class

    # create the built-in pipeline components and add them to the pipeline
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(train_data)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,
                    annotations,
                    drop=0.5, 
                    losses=losses,
                )
            print("Losses", losses)

    # test the model
    for text, _ in train_data[:5]:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        # print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)
    return nlp

In [21]:
labels = ["act","obj",'trm','typ','mtrx']
output_dir="."
n_iter=10
# create_and_train_model(data,output_dir,n_iter)

In [46]:
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

model = spacy.load(output_dir)

# evaluate the model

def evaluate(model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot['entities'])
        pred_value = model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

In [52]:
test_result = evaluate(model, test)
print("F-score - ",test_result['ents_f'],"\n\nResults per type")
test_result['ents_per_type']

F-score -  96.8561872909699 

Results per type


{'obj': {'p': 94.7780678851175, 'r': 98.6413043478261, 'f': 96.67110519307589},
 'act': {'p': 96.40287769784173,
  'r': 98.52941176470588,
  'f': 97.45454545454545},
 'mtrx': {'p': 98.48484848484848, 'r': 100.0, 'f': 99.23664122137404},
 'typ': {'p': 94.01197604790418,
  'r': 99.36708860759494,
  'f': 96.61538461538461},
 'trm': {'p': 62.5, 'r': 100.0, 'f': 76.92307692307693}}

In [27]:
from distractors_generation import get_distractors

print(sentences[1172])
d = get_distractors(sentences[1050:1250], 'those pivots', sentences[1172], model, 6)
print(d)

elimination produces those pivots before the determinant appears
(['rows', 'matrices', 'main diagonals', 'zeroes', 'properties', 'lengths'], ['those ', 'pivots', ''])


In [30]:
from spacy import displacy
displacy.render(model(str(sentences[:100])), jupyter=True, style='ent')